In [2]:
import numpy as np

def kron4(A, B, C, D):
    """Return the Kronecker product A ⊗ B ⊗ C ⊗ D."""
    return np.kron(np.kron(np.kron(A, B), C), D)

def get_identities(delta_A, delta_B, delta_C, delta_D):
    """Return identity matrices for the codomains of delta matrices."""
    return (
        np.eye(delta_A.shape[1]),
        np.eye(delta_B.shape[1]),
        np.eye(delta_C.shape[1]),
        np.eye(delta_D.shape[1])
    )

def construct_HX(delta_A, delta_B, delta_C, delta_D):
    I_f, I_h, I_j, I_l = get_identities(delta_A, delta_B, delta_C, delta_D)

    B11 = kron4(I_f, delta_B, I_j, I_l)
    B12 = kron4(delta_A, I_h, I_j, I_l)
    B21 = kron4(I_f, I_h, delta_C, I_l)
    B23 = B12
    B31 = kron4(I_f, I_h, I_j, delta_D)
    B34 = B12
    B42 = B21
    B43 = B11
    B52 = B31
    B54 = B11
    B63 = B31
    B64 = B21

    dims = [B11.shape[0], B12.shape[1], B23.shape[1], B34.shape[1]]
    Z = lambda r, c: np.zeros((r, c))

    row1 = [B11, B12, Z(dims[0], dims[2]), Z(dims[0], dims[3])]
    row2 = [B21, Z(dims[0], dims[1]), B23, Z(dims[0], dims[3])]
    row3 = [B31, Z(dims[0], dims[1]), Z(dims[0], dims[2]), B34]
    row4 = [Z(dims[0], dims[0]), B42, B43, Z(dims[0], dims[3])]
    row5 = [Z(dims[0], dims[0]), B52, Z(dims[0], dims[2]), B54]
    row6 = [Z(dims[0], dims[0]), Z(dims[0], dims[1]), B63, B64]

    return np.block([row1, row2, row3, row4, row5, row6])

def construct_HZ(delta_A, delta_B, delta_C, delta_D):
    I_f, I_h, I_j, I_l = get_identities(delta_A, delta_B, delta_C, delta_D)

    block = lambda a, b, c, d: kron4(a, b, c, d)
    Z_like = lambda ref: np.zeros_like(ref)

    B01 = block(I_f, I_h, delta_C, I_l)
    B02 = block(I_f, delta_B, I_j, I_l)
    B04 = block(delta_A, I_h, I_j, I_l)
    B07 = block(I_f, I_h, I_j, delta_D)
    B09 = B02
    B011 = B04
    B014 = B07
    B015 = B01
    B018 = B04
    B022 = B07
    B023 = B01
    B024 = B02

    row1 = [B01, B02, Z_like(B01), B04, Z_like(B01), Z_like(B01)]
    row2 = [B07, Z_like(B02), B09, Z_like(B04), B011, Z_like(B04)]
    row3 = [Z_like(B07), B014, B015, Z_like(B04), Z_like(B04), B018]
    row4 = [Z_like(B07), Z_like(B02), Z_like(B015), B022, B023, B024]

    return np.block([row1, row2, row3, row4])

def construct_MX(delta_A, delta_B, delta_C, delta_D):
    I_f, I_h, I_j, I_l = get_identities(delta_A, delta_B, delta_C, delta_D)

    rows = [
        kron4(delta_A, I_h, I_j, I_l),
        kron4(I_f, delta_B, I_j, I_l),
        kron4(I_f, I_h, delta_C, I_l),
        kron4(I_f, I_h, I_j, delta_D),
    ]
    return np.vstack(rows)

def construct_MZ(delta_A, delta_B, delta_C, delta_D):
    I_f, I_h, I_j, I_l = get_identities(delta_A, delta_B, delta_C, delta_D)

    blocks = [
        kron4(I_f, I_h, I_j, delta_D),
        kron4(I_f, I_h, delta_C, I_l),
        kron4(I_f, delta_B, I_j, I_l),
        kron4(delta_A, I_h, I_j, I_l),
    ]
    return np.hstack(blocks)

def construct_4d_matrices(delta_A, delta_B, delta_C, delta_D):
    return (
        construct_HX(delta_A, delta_B, delta_C, delta_D),
        construct_HZ(delta_A, delta_B, delta_C, delta_D),
        construct_MX(delta_A, delta_B, delta_C, delta_D),
        construct_MZ(delta_A, delta_B, delta_C, delta_D)
    )


In [3]:
def random_binary_matrix(rows, cols):
    """Generate a binary matrix with entries in {0, 1}."""
    return np.random.randint(0, 2, size=(rows, cols))

# Dimensions
e, f = 2, 2
g, h = 3, 3
i, j = 2, 2
k, l = 5, 5

# Binary matrices
delta_A = random_binary_matrix(e, f)
delta_B = random_binary_matrix(g, h)
delta_C = random_binary_matrix(i, j)
delta_D = random_binary_matrix(k, l)

hx, hz, mx, mz = construct_4d_matrices(delta_A, delta_B, delta_C, delta_D)